# Adicionando Funções externad utilizando Langchain

Langchain é um framework para criação de aplicações de IA e naturalmente ele possui ferramentas para facilitar a criação de funções externas para serem passadas a api da OpenAI. Para a utilização do LangChain, será necessário entendermos brevemente uma outra biblioteca de Python chamada pydantic, uma biblioteca para validação de dados que facilita a construção de estruturas de dados mais robustas.

In [2]:
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Optional
from enum import Enum

class UnidadeEnum(str, Enum):
  celsius = 'celsius'
  fahrenheit = 'fahrenheit'
  

class ObterTemperaturaAtua(BaseModel):
  """Obtém a temperatura atual de uma determinada localidade"""
  local: str = Field(description='O nome da cidade', examples=['São Paulo', 'Porto Alegre'])
  unidade: Optional[UnidadeEnum]
  

In [3]:
from langchain_core.utils.function_calling import convert_to_openai_function

tool_temperatura = convert_to_openai_function(ObterTemperaturaAtua)
tool_temperatura

{'name': 'ObterTemperaturaAtua',
 'description': 'Obtém a temperatura atual de uma determinada localidade',
 'parameters': {'type': 'object',
  'properties': {'local': {'description': 'O nome da cidade',
    'examples': ['São Paulo', 'Porto Alegre'],
    'type': 'string'},
   'unidade': {'description': 'An enumeration.',
    'enum': ['celsius', 'fahrenheit'],
    'type': 'string'}},
  'required': ['local']}}

## Adicionando função externa utilizando LangChain

Agora que já sabemos criar funções que os modelos de llm entendam, podemos passar essas funções para os modelos de llm através da biblioteca langchain. Para isso temos duas formas, podemos utilizar o parâmetro functions ao chamar o métdoso chat_models.

In [5]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI()

response = chat.invoke('Qual é a temperatura de Sao Paulo', functions=[tool_temperatura])
response

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"local":"São Paulo"}', 'name': 'ObterTemperaturaAtua'}}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 104, 'total_tokens': 127, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-d8f56aa9-f76a-4d0b-9612-d6bc4b6f3c81-0')

Ou podemos dar um bind e criar um novo componente de chat_model que terá acesso a função sempre que for chamado o invoke.
Nestes dois casos o modelo se comportará com o parâmetro "auto" de cahamento de função ou seja, ele chamará a função quando necessitar, caso contrário se comportará como um modelo de linguagem normal.

In [8]:
chat= ChatOpenAI()

chat_com_func = chat.bind(functions=[tool_temperatura])


response = chat_com_func.invoke('Qual é a temperatura de Sao Paulo')
response

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"local":"São Paulo"}', 'name': 'ObterTemperaturaAtua'}}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 104, 'total_tokens': 127, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-415f3df0-8b54-4fe7-afe0-9080028c3798-0')

Podemos obrigar o modelo a sempre chamar uma função da seguinte forma:

In [9]:

response = chat.invoke(
  'Qual é a temperatura de Sao Paulo', 
  functions=[tool_temperatura],
  function_call={'name': 'ObterTemperaturaAtua'}
  )
response

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"local":"São Paulo"}', 'name': 'ObterTemperaturaAtua'}}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 119, 'total_tokens': 127, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-badbad67-7c95-4ae9-b3bb-062413c047df-0')

In [10]:
response = chat.invoke(
  'Olá', 
  functions=[tool_temperatura],
  function_call={'name': 'ObterTemperaturaAtua'}
  )
response

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"local":"São Paulo"}', 'name': 'ObterTemperaturaAtua'}}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 114, 'total_tokens': 122, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-7049fdeb-1a87-454b-b4ba-93dbf8e2baab-0')

## Adicionando a uma chain

Podemos adicionar agora este modelo com funções a um prompt e criar uma chain.

In [13]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
  ('system', 'Você é um assistente amigável chamado Jeremias'),
  ('user', '{input}')
])

chain = prompt | chat.bind(functions=[tool_temperatura])

In [14]:
chain.invoke({'input': 'Olá'})

AIMessage(content='Olá! Como posso ajudar você hoje?', response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 113, 'total_tokens': 126, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-2907a509-2fb3-4b48-a685-d6b9aec6fa48-0')

In [15]:
chain.invoke({'input': 'Qual a temperatura em Itanhaém'})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"local":"Itanhaém"}', 'name': 'ObterTemperaturaAtua'}}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 119, 'total_tokens': 143, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-edc0ef2b-b907-4f37-9d50-31031d2470a1-0')